In [1]:
import pandas as pd
import numpy as np
import re

**Combining Datasets Together**

In [3]:
start_year = 2016
end_year = 2023
folder_names = np.array([[f'{k}Q{i}_cf' for i in range(1, 5)] for k in range(start_year, end_year+1)]).flatten()

In [4]:
folder_names[1:]

array(['2016Q2_cf', '2016Q3_cf', '2016Q4_cf', '2017Q1_cf', '2017Q2_cf',
       '2017Q3_cf', '2017Q4_cf', '2018Q1_cf', '2018Q2_cf', '2018Q3_cf',
       '2018Q4_cf', '2019Q1_cf', '2019Q2_cf', '2019Q3_cf', '2019Q4_cf',
       '2020Q1_cf', '2020Q2_cf', '2020Q3_cf', '2020Q4_cf', '2021Q1_cf',
       '2021Q2_cf', '2021Q3_cf', '2021Q4_cf', '2022Q1_cf', '2022Q2_cf',
       '2022Q3_cf', '2022Q4_cf', '2023Q1_cf', '2023Q2_cf', '2023Q3_cf',
       '2023Q4_cf'], dtype='<U9')

In [5]:
# relevant file names
file_name_submission = 'FORM_C_SUBMISSION.csv'
file_name_issuer_info = 'FORM_C_ISSUER_INFORMATION.csv'
file_name_disclosure = 'FORM_C_DISCLOSURE.csv'
file_name_issuer_signature = 'FORM_C_ISSUER_SIGNATURE.csv'
file_names = [file_name_issuer_info, file_name_disclosure, file_name_issuer_signature] # exclude SUBMISSION.csv

start_year = 2016
end_year = 2023
folder_names = np.array([[f'{k}Q{i}_cf' for i in range(1, 5)] for k in range(start_year, end_year+1)]).flatten()
folder_names = folder_names[1:] # Drop 2016Q1_cf because it does not exist

file_path = 'sec_dataset/{}/{}'

# Vertically union between files
df = pd.DataFrame()
for folder_name in folder_names:
    # Horizontally join between files, of the same quarter
    df_hor = pd.read_csv(file_path.format(folder_name, file_name_submission))
    for file_name in file_names:
        df_temp = pd.read_csv(file_path.format(folder_name, file_name))
        # perform left-join
        df_hor = pd.merge(left=df_hor, right=df_temp, on='ACCESSION_NUMBER', how='left')
    
    # perform union
    df = pd.concat([df, df_hor], ignore_index=True)

**Ensure each column has the correct data type**

In [7]:
df_cleaned = df.copy()

In [8]:
# FILING_DATE -> From int64 to Date
df_cleaned['FILING_DATE'] = pd.to_datetime(df['FILING_DATE'].astype(str), format='%Y%m%d')

# DATEINCORPORATION -> From object to Date
df_cleaned['DATEINCORPORATION'] = pd.to_datetime(df['DATEINCORPORATION'].astype(str))

# OVERSUBSCRIPTIONACCEPTED -> From object to integer (where Y and N become 1 and 0, respectively)
df_cleaned['OVERSUBSCRIPTIONACCEPTED'] = df['OVERSUBSCRIPTIONACCEPTED'].map({'Y':1, 'N':0})

# DEADLINEDATE -> From object to Date
df_cleaned['DEADLINEDATE'] = pd.to_datetime(df['DEADLINEDATE'].astype(str), format='%Y-%m-%d')

In [9]:
# check data type
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25436 entries, 0 to 25435
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   ACCESSION_NUMBER                25436 non-null  object        
 1   SUBMISSION_TYPE                 25436 non-null  object        
 2   FILING_DATE                     25436 non-null  datetime64[ns]
 3   CIK                             25434 non-null  float64       
 4   FILE_NUMBER                     25436 non-null  object        
 5   PERIOD                          3400 non-null   float64       
 6   ISAMENDMENT                     25436 non-null  int64         
 7   PROGRESSUPDATE                  3948 non-null   object        
 8   NATUREOFAMENDMENT               8543 non-null   object        
 9   NAMEOFISSUER                    25434 non-null  object        
 10  LEGALSTATUSFORM                 24501 non-null  object        
 11  LE

**Drop uneccessary columns**

In [11]:
# Drop PERIOD
df_cleaned.drop('PERIOD', inplace=True, axis=1)

**Filtering**: To only include relevant filings
- Only include the following submission forms: C, C/A, C-U, C-W, C/A-W, C-U-W

In [13]:
# Only include the following submission forms: C, C/A, C-U, C-W, C/A-W, C-U-W
submission_types = ['C', 'C/A', 'C-U', 'C-W', 'C/A-W', 'C-U-W']
df_filtered = df_cleaned.loc[df_cleaned['SUBMISSION_TYPE'].isin(submission_types), :]

In [14]:
# To oraganize, sort by FILE_NUMBER and FILING_DATE
df_filtered.sort_values(by=['FILE_NUMBER', 'FILING_DATE', 'SUBMISSION_TYPE'], ascending=[True, True, True], inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_25156\1178657395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.sort_values(by=['FILE_NUMBER', 'FILING_DATE', 'SUBMISSION_TYPE'], ascending=[True, True, True], inplace=True)


**Filtering**: To only include relevant filings
- This will be completed much later -> Only include CF offerings that start between 2018 Q1 and 2023 Q4
- This is completed below -> Only include CF offerings thst end between 2018 Q1 and 2023 Q4

In [16]:
df_filtered.columns

Index(['ACCESSION_NUMBER', 'SUBMISSION_TYPE', 'FILING_DATE', 'CIK',
       'FILE_NUMBER', 'ISAMENDMENT', 'PROGRESSUPDATE', 'NATUREOFAMENDMENT',
       'NAMEOFISSUER', 'LEGALSTATUSFORM', 'LEGALSTATUSOTHERDESC',
       'JURISDICTIONORGANIZATION', 'DATEINCORPORATION', 'STREET1', 'STREET2',
       'CITY', 'STATEORCOUNTRY', 'ZIPCODE', 'ISSUERWEBSITE', 'COMPANYNAME',
       'COMMISSIONCIK', 'COMMISSIONFILENUMBER', 'CRDNUMBER',
       'COMPENSATIONAMOUNT', 'FINANCIALINTEREST', 'SECURITYOFFEREDTYPE',
       'SECURITYOFFEREDOTHERDESC', 'NOOFSECURITYOFFERED', 'PRICE',
       'PRICEDETERMINATIONMETHOD', 'OFFERINGAMOUNT',
       'OVERSUBSCRIPTIONACCEPTED', 'OVERSUBSCRIPTIONALLOCATIONTYPE',
       'DESCOVERSUBSCRIPTION', 'MAXIMUMOFFERINGAMOUNT', 'DEADLINEDATE',
       'CURRENTEMPLOYEES', 'TOTALASSETMOSTRECENTFISCALYEAR',
       'TOTALASSETPRIORFISCALYEAR', 'CASHEQUIMOSTRECENTFISCALYEAR',
       'CASHEQUIPRIORFISCALYEAR', 'ACTRECEIVEDRECENTFISCALYEAR',
       'ACTRECEIVEDPRIORFISCALYEAR', 'SHORTTERM

In [17]:
# Ensure to only include CF offerings thst end between 2018 Q1 and 2023 Q4
all_file_numbers = df_filtered['FILE_NUMBER'].unique() # list of all unique FILE_NUMBER values
df_filtered['FINALDEADLINEDATE'] = np.nan # store the offering's latest deadline -> to be used later in feature engineering # of CF competitors

for file_number in all_file_numbers:
    selected = df_filtered.loc[df_filtered['FILE_NUMBER'] == file_number,:]
    
    # In the case there are updates on offering terms
    if (selected['SUBMISSION_TYPE'] == 'C/A').any():
        # Obtain deadline from the latest updated offering terms
        deadline = selected.loc[selected['SUBMISSION_TYPE'] == 'C/A', :].iloc[-1,:]['DEADLINEDATE']
    # In the case that there is no update on offering terms
    else:
        # The most up-to-date deadline is from the original Form C submission
        deadline = selected.loc[selected['SUBMISSION_TYPE'] == 'C', :].iloc[0,:]['DEADLINEDATE']
    
    # Drop offerings that ends beyond 2023 Q4
    if deadline >= pd.Timestamp('2023-12-30'):
        df_filtered.drop(index=selected.index, inplace=True)
    
    # Store the offering's latest deadline -> to be used later in feature engineering # of CF competitors
    df_filtered.loc[df_filtered['FILE_NUMBER'] == file_number, 'FINALDEADLINEDATE'] = deadline

C:\Users\User\AppData\Local\Temp\ipykernel_25156\1883406018.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['FINALDEADLINEDATE'] = np.nan # store the offering's latest deadline -> to be used later in feature engineering # of CF competitors
C:\Users\User\AppData\Local\Temp\ipykernel_25156\1883406018.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[df_filtered['FILE_NUMBER'] == file_number, 'FINALDEADLINEDATE'] = deadline
C:\Users\User\AppData\Local\Temp\ipykernel_25156\188

**Define Target Variable**
- 1 and 0 represents successful and failed CF offering, respectively.
- Definition of successful CF offering. There are 2 definitions,
    - First definition: In the case that the \$ amount raised is disclosed (Done)
        - Ensure that we look at the last C-U filing for a given FILE_NUMBER
        - Obtain the \$ amount raised from '`PROGRESSUPDATE` using regular expression
        - Compare the \$ amount raised with the `OFFERINGAMOUNT`
    - Second definition: In the case that the \$ amount is not disclosed (Done)
        - Ensure that we look at the last C-U filing for a given FILE_NUMBER
        - `PROGRESSUPDATE` contains the word '100%' -> use regular expression to allow any symbol between '100' and '%'
- Definition of failed CF offering.
    - First definition: In the case that C-U form is not provided. (Done)
    - Second definition: C-U form is provided, but the disclosed amount raised is 0 or less than the targeted amount. (Done)
    - Third definition: If the Form C-W is filed. (Done)
        - Those that filed C-W are usually the ones that fail to meet the offering target, and so when C-W is filed, it will be considered as a failed CF.

Relevant for the `extract_offering_raised` function,
- If there are more than one \$ amount mentioned, pick the largest one. For example, "On November 30th 2019 Aapoon, Inc raised `\$102.100` against the goal of raising `\$100,000`".
    - Examples:
        - "The issuer closed on $19,300 of SAFEs. The funding portal was granted $380 of securities as compensation. Raise Green was granted $1,000 in additional compensation for a Form C/A Material Change."

Regex rule for if the $ amount raised was not disclosed -> relevant for the `extract_text_outcome` function. 3 Options:
- Success
    - What should be in the regex?
        - Make sure to lower case all the letters in the PROGRESSUPDATE
        - There is no "not" word in the description
        - "success", "successfully"
        - "100%", "100 percent"
        - "reach", "reached"
        - "minimum"
        - "met"
        - "full", "fully"
        - "exceed", "exceeded"
    - Examples:
        - "The Offering closed successfully on 8/06/2021, having received investment commitments of 90,040.00"
        - "Initial closed updated for a successfully closed offering."
        - "100% of minimum target reached"
        - "Minimum Funding Goal has been met."
        - "Minimum target amount raised"
        - "100% of target amount raised by target date. Offering closed"
        - "100% of Target Goal reached Target date reached and offering closed"
        - "Project fully funded."
        - "Max target reached."
        - "Project has reached the Target Offering Amount withing the Offering Period."
        - "Reached the target and almost the max offering amount."
        - "Target Amount in the Form C reached, Issuer intends to engage in initial close and collection of committed funds and then a subsequent re-opening of its offering."
        - "Luma Resources LLC is has received investment committments and has funds in escrow that are in excess of 50 percent and 100 percent of the target offering amount."
        - "The offer exceeded its original goal with 220 investors 349,000 shares sold. The will continue its raise until it reach 107,000."
        
- Fail
    - What should be in the regex?
        - Make sure to lower case all the letters in the PROGRESSUPDATE
        - "fail", "failed"
        - "return", "returned"
        - "refund", "refunded"
        - "The company did not have any sales"
        - "No securities were sold in this offering"
        - "No investments were sold"
        - "did not sell"
        - "not"
        - "no sales"
        - The word "50%" is in the desription, but the word "100%" is not
    - Examples:
        - "End of offering. Issuer failed to reach target amount and investor funds are being returned by the escrow agent."
        - "THE OFFERING DID NOT MEET TARGET; FUNDS BEING RETURNED TO INVESTORS."
        - "The company did not have any sales in this offering."
        - "50% of target amount raised."
        - "The offering did not meet its target; funds being returned to investors."
        - "The issuer failed to meet the minimum target amount in the offering. All funds committed by investors have been refunded."
        - "The raise did not meet its minimum target amount and thus was not successful. Investor transaction commitments are being canceled and monies are being refunded."
        - "No securities were sold in this offering."
        - "The Company did not sell any securities under Regulation CF"
        - "Offering closed.  Target amount not reached" -> this appears quite often
        - "Offering closed.  Target amount was not reahed" -> yes it is a typo
        - "The issuer did not sell any securities or raise any capital in this offering prior to its deadline."
        - "Update filing for termination of offering.  No investments were sold in this offering."
        - "The offering is closed and the company has made no sales."
        - "The raise did not meet its minimum target amount and thus was not successful. Investor transaction commitments are being canceled and monies are being refunded."
        
- Remove from dataset
    - What should be in the regex?
        - Make sure to lower case all the letters in the PROGRESSUPDATE
    - Examples:
        - "Issuer is launching campaign with StartEngine"
        - "To disclose related material events; which originated on April 30, 2021 when the Issuer's Portal terminated trading as noted in the CUW.  The Issuer has replaced the former Portal with truCrowd and hereby amends the CUW file to continue the Offering."
        - "The Form C-U is amended to list the correct offering deadline of July 21, 2020."
        - "Three43 Inc. has added a new video (Video 2) further describing their product."
        - "The Offering closed on 9/21/2023 due to the untimely closing, withdrawal, of the Broker Dealer of Record."
        - "Offer ended on 4/01/2019 with 138 investors."
        - "The offering closed December 31,2021 "
        - "Completion of Offering"
    
- Manually classify -> I decided to just remove these data points
    - Examples:
        - "At the close of the offering, the issuer closed on 106,787.52 and 228,860 number of securities."

In [20]:
def extract_offering_raised(progress_update_text):
    '''
    Uses regular expression to extract the $ amount of offering raised from PROGRESSUPDATE column 
    '''    
    # Use regex to extract amount raised
    pattern = r'\$\s*\d+(?:,\d{3})*(?:\.\d{2})?' # r'\$\d{1,3}(?:,\d{3})*(?:\.\d{1,2})?|\$\d+(?:\.\d{1,2})?'
    matches = re.findall(pattern, progress_update_text)
    
    # In the case that no $ amount raised was disclosed
    if not matches:
        return np.nan
    
    # convert from string to float
    amount_raised = []
    for match in matches:
        amount_raised.append(float(match[1:].replace(',', '')))
    
    # in the case that there are multiple $ figures, pick the largest one
    result = max(amount_raised)
    
    return result

In [21]:
def extract_text_outcome(progress_update_text):
    '''
    Use regex to extract on whether the CF is a success, failure, or should be omitted from the dataset, in the case that
    $ amount raised was not disclosed in the latest C-U filing.
    Returns:
        'success' -> Successful CF
        'fail' -> Unsuccessful CF
        'uncertain' -> Not classified (to be removed from the training dataset)
    '''
    text = progress_update_text.lower()

    # Failure patterns
    fail_patterns = [
        r'\bfail(?:ed)?\b',
        r'\breturn(?:ed)?\b',
        r'\brefund(?:ed)?\b',
        r'did not have any sales',
        r'no securities were sold in this offering',
        r'no investments were sold',
        r'did not sell',
        r'\bno sales\b',
        r'\bnot\b'
    ]
    
    # Special case: 50% present without 100%
    if re.findall(r'\b50\s*%|\b50\s*percent\b', text) and not re.findall(r'\b100\s*%|\b100\s*percent\b', text):
        return 'fail'
    
    if any(re.findall(p, text) for p in fail_patterns):
        return 'fail'
    
    # Success patterns
    # Must not contain the word "not"
    if 'not' in text:
        return 'fail'

    success_patterns = [
        r'\bsuccess(?:fully)?\b',
        r'\b100\s*%|\b100\s*percent\b',
        r'\breach(?:ed)?\b',
        r'\bminimum\b',
        r'\bmet\b',
        r'\bfull(?:y)?\b',
        r'\bexceed(?:ed)?\b'
    ]

    if any(re.findall(p, text) for p in success_patterns):
        return 'success'    

    # If no match
    return 'uncertain'

Note that the cell below, I am defining the amount raised and final target offering amount, which will be used later to define target variable in the next 2 cell.

The general rule in defining the amount raised is as follows:
- If the \$ amount raised is disclosed, that will be used as the amount raised.
- If the \$ amount raised is not disclosed, but the progress update describe whether the outcome is success or failure: (i) if it indicates success, the amount raised is assumed to be equal to the final target offering amount; (ii) if it indicates failure, the amount raised is assumed to be zero
- If there is no C-U filed, the amount raised is assume to be zero.

In [23]:
all_file_numbers = df_filtered['FILE_NUMBER'].unique() # list of all unique FILE_NUMBER values

# Store updated dataset
df_filtered_2 = df_filtered.copy()
df_filtered_2['FINALOFFERINGTARGET'] = np.nan # create new column to store the final offering target amount
df_filtered_2['AMOUNTRAISED'] = np.nan # create new column to store $ amount raised
df_filtered_2['Y'] = np.nan # create a new column to store the target variable

# Copmute total amount raised
for file_number in all_file_numbers:
    dropped_idx = []
    # Select the offering
    selected = df_filtered.loc[df_filtered['FILE_NUMBER'] == file_number, :]
    
    # Extract the final progress update in the latest C-U filing, if any
    progress_update = np.nan
    for i in range(-1, len(selected)*-1 - 1, -1):
        if selected.iloc[i, :]['SUBMISSION_TYPE'] == 'C-U':
            progress_update = selected.iloc[i, :]['PROGRESSUPDATE']
            break
        else:
            continue
            
    # extract the latest target offering amount
    final_target = np.nan
    for i in range(-1, len(selected)*-1 - 1, -1):
        if pd.isna(final_target):
            final_target = selected.iloc[i, :]['OFFERINGAMOUNT']
        else:
            break
    df_filtered_2.loc[selected.index, 'FINALOFFERINGTARGET'] = final_target
            
    # if there is no C-U filing, i.e. no progress update
    if pd.isna(progress_update):
        df_filtered_2.loc[selected.index, 'AMOUNTRAISED'] = 0 # set amount raised to 0
        continue
    # otherwise, extract the amount raised from progress_update
    else:
        amount_raised = extract_offering_raised(progress_update)

    # if `amount_raised` is not null
    if not pd.isna(amount_raised):
        # update the AMOUNTRAISED column
        df_filtered_2.loc[selected.index, 'AMOUNTRAISED'] = amount_raised
    # if `amount_raised` is null
    else:
        regex_classification = extract_text_outcome(progress_update)
        # set AMOUNTRAISED equal to the final_target amount
        if regex_classification == 'success':
            df_filtered_2.loc[selected.index, 'AMOUNTRAISED'] = final_target
        # set AMOUNTRAISED to 0
        elif regex_classification == 'fail':
            df_filtered_2.loc[selected.index, 'AMOUNTRAISED'] = 0
        # remove unclassified datapoints
        else:
            # print(progress_update) ##############################
            df_filtered_2.drop(index=selected.index, inplace=True)
            dropped_idx = selected.index.to_list()
    
    # Special case: When C-W or C/A-W is filed
    for i in range(-1, len(selected)*-1 - 1, -1):
        if selected.iloc[i, :]['SUBMISSION_TYPE'] == 'C/A-W':
            break
        elif not dropped_idx and selected.iloc[i, :]['SUBMISSION_TYPE'] == 'C-W':
            df_filtered_2.loc[selected.index, 'AMOUNTRAISED'] = 0
            break

Before we continue, I noticed that there is an entry error of file number '020-32053', where it has no information on OFFERINGAMOUNT, so we are going to remove it.

In [25]:
# print offering file number '020-32053'
cols = ['ACCESSION_NUMBER', 'SUBMISSION_TYPE', 'FILING_DATE', 'CIK', 'FILE_NUMBER', 'PROGRESSUPDATE', 'NATUREOFAMENDMENT', 'DEADLINEDATE', 'OFFERINGAMOUNT', 'AMOUNTRAISED', 'FINALOFFERINGTARGET']
anomaly_cf = df_filtered_2.loc[df_filtered_2['FILE_NUMBER'] == '020-32053', cols]
display(anomaly_cf)

# drop the anomaly cf
df_filtered_2.drop(index=anomaly_cf.index, inplace=True)

,ACCESSION_NUMBER,SUBMISSION_TYPE,FILING_DATE,CIK,FILE_NUMBER,PROGRESSUPDATE,NATUREOFAMENDMENT,DEADLINEDATE,OFFERINGAMOUNT,AMOUNTRAISED,FINALOFFERINGTARGET
20627,0001376474-23-000168,C,2023-03-27,NaN,020-32053,NaN,NaN,NaT,NaN,0.0,NaN
20612,0001376474-23-000174,C-W,2023-03-28,NaN,020-32053,NaN,NaN,NaT,NaN,0.0,NaN


Now, we can use the `AMOUNTRAISED` and `FINALOFFERINGTARGET` that we defined previously to determine whether the fundraising is successful or not.

In [27]:
# Define the target variable
df_filtered_2['Y'] = (df_filtered_2['AMOUNTRAISED'] >= df_filtered_2['FINALOFFERINGTARGET']).replace({True:1, False:0})

# Feature Engineering
- Note that we are creating a ML model to predict the success/failure of CF offering at the start date of the CF offering. Therefore, for the features, we will be using information up to the start date of the CF offerring.

In [29]:
ml_df = pd.DataFrame({'FILE_NUMBER': df_filtered_2['FILE_NUMBER'].unique()})
ml_df.set_index('FILE_NUMBER', inplace=True)

# Add the target variable
ml_df['Y'] = np.nan
for num in df_filtered_2['FILE_NUMBER'].unique():
    selected = df_filtered_2.loc[df_filtered_2['FILE_NUMBER'] == num, :]
    for i in range(len(selected)):
        if selected.iloc[i, :]['SUBMISSION_TYPE'] == 'C':
            ml_df.loc[num, 'Y'] = selected.iloc[i, :]['Y']
            break

### Transfer Features from `df_filtered_2` to `ml_df`

In [31]:
def extract_latest_variable(col_df_filtered, col_ml_df):
    '''
    This function is used to extract the latest information on a variable of a CF offering, transfering the information from
    the df_filtered_2 to ml_df.
    Parameters
    ----------
    col_df_filtered: str
        The feature name from the df_filtered_2 dataset that you want to extract.
    col_ml_df: str
        The feature name that you want your feature to be called in ml_df.
    
    Returns
    -------
    It does not return anything. It automatically updates the ml_df dataset.
    '''
    all_file_numbers = ml_df.index.unique() # list of all unique FILE_NUMBER values
    ml_df[col_ml_df] = np.nan # store the offering's latest information

    for file_number in all_file_numbers:
        selected = df_filtered_2.loc[df_filtered_2['FILE_NUMBER'] == file_number,:]

        # In the case there are updates on offering terms
        if (selected['SUBMISSION_TYPE'] == 'C/A').any():
            # Obtain deadline from the latest updated offering terms
            var = selected.loc[selected['SUBMISSION_TYPE'] == 'C/A', :].iloc[-1,:][col_df_filtered]
        # In the case that there is no update on offering terms
        else:
            # The most up-to-date deadline is from the original Form C submission
            var = selected.loc[selected['SUBMISSION_TYPE'] == 'C', :].iloc[0,:][col_df_filtered]

        # Store the offering's latest deadline -> to be used later in feature engineering # of CF competitors
        ml_df.loc[file_number, col_ml_df] = var

In [32]:
def extract_filetered_df_to_ml_df(col_df_filtered, col_ml_df):
    '''
    This function is used to extract a feature from df_filtered_2 to ml_df. More specifically, it extracts features
    from the intial form C submission in the df_filtered_2.
    
    Parameters
    ----------
    col_df_filtered: str
        The feature name from the df_filtered_2 dataset that you want to extract.
    col_ml_df: str
        The feature name that you want your feature to be called in ml_df.
    
    Returns
    -------
    It does not return anything. It automatically updates the ml_df dataset.
    '''
    ml_df[col_ml_df] = np.nan
    for num in df_filtered_2['FILE_NUMBER'].unique():
        selected = df_filtered_2.loc[df_filtered_2['FILE_NUMBER'] == num, :]
        for i in range(len(selected)):
            if selected.iloc[i, :]['SUBMISSION_TYPE'] == 'C':
                ml_df.loc[num, col_ml_df] = selected.iloc[i, :][col_df_filtered]
                break

In [33]:
# Extract revenue figure from the most recent fiscal year
extract_filetered_df_to_ml_df('REVENUEMOSTRECENTFISCALYEAR', 'REVENUE')

In [34]:
# Extract revenue figure from the previous fiscal year
extract_filetered_df_to_ml_df('REVENUEPRIORFISCALYEAR', 'PREVIOUSREVENUE')

In [35]:
# Extract total asset from the current fiscal year
extract_filetered_df_to_ml_df('TOTALASSETMOSTRECENTFISCALYEAR', 'ASSET')

In [36]:
# Extract cash and cash-equivalents from the current fiscal year
extract_filetered_df_to_ml_df('CASHEQUIMOSTRECENTFISCALYEAR', 'CASH')

In [37]:
# Extract long-term debt from the current fiscal year
extract_filetered_df_to_ml_df('LONGTERMDEBTRECENTFISCALYEAR', 'LONGTERMDEBT')

In [38]:
# Extract net income from the current fiscal year
extract_filetered_df_to_ml_df('NETINCOMEMOSTRECENTFISCALYEAR', 'NETINCOME')

In [39]:
# Extract short-term debt from the current fiscal year
extract_filetered_df_to_ml_df('SHORTTERMDEBTMRECENTFISCALYEAR', 'SHORTTERMDEBT')

In [40]:
# Extract Acc. Rec. from the current fiscal year
extract_filetered_df_to_ml_df('ACTRECEIVEDRECENTFISCALYEAR', 'ACCREC')

In [41]:
# Extract COGSfrom the current fiscal year
extract_filetered_df_to_ml_df('COSTGOODSSOLDRECENTFISCALYEAR', 'COGS')

In [42]:
# Add minimum investiment size feature
extract_filetered_df_to_ml_df('PRICE', 'MINIMUMSIZE')

In [43]:
# Add the commission in text format
extract_filetered_df_to_ml_df('COMPENSATIONAMOUNT', 'TEXT_RAISEDCOMMISSION')

In [44]:
# Add the commission in text format
extract_filetered_df_to_ml_df('FINANCIALINTEREST', 'TEXT_EQUITYCOMMISSION')

In [45]:
# Add the platform ID
extract_filetered_df_to_ml_df('COMMISSIONCIK', 'COMMISSIONCIK')

In [46]:
# Add the offering start date
extract_filetered_df_to_ml_df('FILING_DATE', 'FILING_DATE')

In [47]:
# Add the offering deadline
extract_filetered_df_to_ml_df('FINALDEADLINEDATE', 'FINALDEADLINEDATE')

In [48]:
# Compute the target offering amount
extract_latest_variable('OFFERINGAMOUNT', 'FINALOFFERINGAMOUNT')

In [49]:
# Obtain the type of security offered in the CF
extract_latest_variable('SECURITYOFFEREDTYPE', 'SECURITYOFFEREDTYPE')

In [50]:
# Obtain further description for "Others" security type
extract_latest_variable('SECURITYOFFEREDOTHERDESC', 'SECURITYOFFEREDOTHERDESC')

In [51]:
# Extract PRICEDETERMINATIONMETHOD
extract_filetered_df_to_ml_df('PRICEDETERMINATIONMETHOD', 'PRICEDETERMINATIONMETHOD')

In [52]:
# Obtain the date when the company was found
extract_filetered_df_to_ml_df('DATEINCORPORATION', 'DATEINCORPORATION')

In [53]:
# Obtain the issuer ID
extract_filetered_df_to_ml_df('CIK', 'CIK')

In [54]:
# Obtain the issuing person's name
extract_filetered_df_to_ml_df('ISSUERSIGNATURE', 'ISSUERSIGNATURE')

In [55]:
# Obtain the number of employees of the issuer
extract_filetered_df_to_ml_df('CURRENTEMPLOYEES', 'CURRENTEMPLOYEES')

In [56]:
# Obtain the organization form of the issuer
extract_filetered_df_to_ml_df('LEGALSTATUSFORM', 'LEGALSTATUSFORM')

In [57]:
# Obtain the organization form description of the issuer
extract_filetered_df_to_ml_df('LEGALSTATUSOTHERDESC', 'LEGALSTATUSOTHERDESC')

In [58]:
# Extract the state in which the company is registered / incorporated
extract_filetered_df_to_ml_df('JURISDICTIONORGANIZATION', 'JURISDICTIONORGANIZATION')

In [59]:
# Extract the state in which the company is currently operating
extract_filetered_df_to_ml_df('STATEORCOUNTRY', 'STATEORCOUNTRY')

### Cleaning The ml_df Dataset (Part 1)

The following will be performed:
- Exclude data points where it reports 0 cash.
- Exclude data points where it simulateneosly reports 0 revenue and non-zero COGS.
- Exclude data points where it simulateneosly reports non-zero revenue and 0 COGS.
- Exclude data points where it simultaneously reports 0 revenue, 0 COGS, 0 net income, and the company has employees on record (which they would have to pay, i.e. wages expense).
- Only include data points where the issuer operates in one of the 50 US states, excluding  regions such as DC (DISTRICT OF COLUMBIA), PR (PUERTO RICO), VI (VIRGIN ISLANDS), etc.

In [137]:
# save ml_df just in case
ml_df_copy = ml_df.copy()

In [248]:
# Apply filter to exclude companies that have 0 cash
mask = (ml_df['CASH'] == 0)
ml_df = ml_df.loc[~mask, :].copy()

In [250]:
# Apply filter to exclude companies that simulateneosly reports 0 revenue and non-zero COGS.
mask = (ml_df['REVENUE'] == 0) & (ml_df['COGS'] != 0)
ml_df = ml_df.loc[~mask, :].copy()

In [252]:
# Apply filter to exclude companies that simulateneosly reports non-zero revenue and 0 COGS.
mask = (ml_df['REVENUE'] != 0) & (ml_df['COGS'] == 0)
ml_df = ml_df.loc[~mask, :].copy()

In [254]:
# Apply filter to exclude companies that simultaneously reports 0 revenue, 0 COGS, 0 net income, and the company has employees on record
mask = (ml_df['REVENUE'] == 0) & (ml_df['COGS'] == 0) & (ml_df['NETINCOME'] == 0)
ml_df = ml_df.loc[~mask, :].copy()

In [256]:
# Apply filter to only include companies that are operating in the USA
us_states = {
    'AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','IA','ID','IL','IN',
    'KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH',
    'NJ','NM','NV','NY','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VA',
    'VT','WA','WI','WV','WY'
}

mask = ml_df['STATEORCOUNTRY'].isin(us_states)
ml_df = ml_df.loc[mask, :].copy()

### Historical Financial Features

Revenue Feature

In [260]:
# Correct any misinputs
ml_df['REVENUE'] = ml_df['REVENUE'].abs()

Revenue Growth Feature

In [263]:
# Correct any misinputs
ml_df['PREVIOUSREVENUE'] = ml_df['PREVIOUSREVENUE'].abs()

In [265]:
# Compute Revenue Log Growth
ml_df['REVENUEGROWTH'] = np.log(ml_df['REVENUE'] + 1) - np.log(ml_df['PREVIOUSREVENUE'] + 1)

Asset Feature

In [267]:
# Correct any misinputs
ml_df['ASSET'] = ml_df['ASSET'].abs()

Cash Feature

In [270]:
# Correct any misinputs
ml_df['CASH'] = ml_df['CASH'].abs()

Long-Term Debt Feature

In [273]:
# Correct any misinputs
ml_df['LONGTERMDEBT'] = ml_df['LONGTERMDEBT'].abs()

Debt-to-Asset Ratio Feature

In [275]:
# Correct any misinputs
ml_df['SHORTTERMDEBT'] = ml_df['SHORTTERMDEBT'].abs()

In [277]:
# Compute debt-to-asset ratio
ml_df['DEBTTOASSET'] = np.arcsinh((ml_df['LONGTERMDEBT'] + ml_df['SHORTTERMDEBT']) / (ml_df['ASSET'] + 1e-10))

Working Capital Feature

In [280]:
# Correct any misinputs
ml_df['ACCREC'] = ml_df['ACCREC'].abs()

In [282]:
# Compute working capital
ml_df['WORKINGCAPITAL'] = ml_df['CASH'] + ml_df['ACCREC'] - ml_df['SHORTTERMDEBT']

Return on Asset (ROA) Ratio Feature

In [285]:
# Compute ROA
ml_df['ROA'] = np.arcsinh(ml_df['NETINCOME'] / (ml_df['ASSET'] + 1e-10))

Gross Profit Feature

In [287]:
# Correct any misinputs
ml_df['COGS'] = ml_df['COGS'].abs()

In [289]:
# Compute gross profit
ml_df['GROSSPROFIT'] = ml_df['REVENUE'] - ml_df['COGS']

Gross Margin Feature

In [292]:
# Compute gross margin
ml_df['GROSSMARGIN'] = np.arcsinh((ml_df['REVENUE'] - ml_df['COGS']) / (ml_df['REVENUE'] + 1e-10))

Net Profit Margin Feature

In [295]:
# Compute gross margin
ml_df['NETPROFITMARGIN'] = np.arcsinh(ml_df['NETINCOME'] / (ml_df['REVENUE'] + 1e-10))

### Crowdfunding Characteristics Features

Platform Fee (commission based on amount raised) Feature

After Platform % Fee based on amount raised has been extracted, we use the following rule set:
- If there is only a single percentage, we just use that as the Platform % Fee based on amount raised.
- If there are more than three commissions extracted: For clarity, look at the following example, given "Applied at marginal rate based upon amount of total offering: up to $50,000 = 8.0%, $50,001 - $100,000 = 7.0%, $100,001+ = 6.0%. $250 posting fee. 2.85% investment fee capped at $37.25.", we would have extracted [8.0, 7.0, 6.0, 2.85]. Since know that we should average the [8.0, 7.0, 6.0] and add the 2.85 to the average., we take the following steps,
    - Order the list in descending order.
    - If the difference between consecutive sequence is less than or equal to 2, we average them; Otherwise, we will add the number to the average.

Regex for COMPENSATIONAMOUNT Examples:

Contains % commission only
- "The intermediary will get paid a success fee of 9%."
- "The intermediary will get paid a success fee of 7%"
- "6 percent"
- "7 - 13 percent"
- "7.0 percent"
- "MainVest will be paid 6.0% of the final offering amount, upon the successful completion of the offering. MainVest does not receive compensation if the offering does not succeed."
- "<p>6.5% of the offering amount upon a successful fundraise, and be entitled to reimbursement for out-of-pocket third party expenses it pays or incurs on behalf of the Issuer in connection with the offering.</p>"
- "At the conclusion of the offering, the issuer shall pay a fee of six percent (6%) of the amount raised in the offering to the Intermediary."
- "7.5% of the offering amount upon a successful fundraise, and be entitled to reimbursement for out-of-pocket third party expenses it pays or incurs on behalf of the Issuer in connection with the offering."
- "MainVest will be paid Three (3) Percent of the amount of the Offering raised by "In-Network Users" of the Platform plus Nine (9) Percent of the amount of the Offering raised by all other investors."
- "MainVest will be paid 6.0% of the final offering amount, upon the successful completion of the offering. MainVest does not receive compensation if the offering does not succeed."

Tiered fee structure
- "Applied at marginal rate based upon amount of total offering: up to$50,000 = 8.0%, $50,001 - $100,000 = 7.0%, $100,001+ = 6.0%. $250 posting fee. 2.85% investment fee capped at $37.25."
- "The intermediary collects a 5% fee on the first $1 million raised. 3% of anything raised over $1 million"
- 10% of the total Offering Amount; provided only 5% is charged on investors introduced by the issuer.
- "MainVest will be paid Three (3) Percent of the amount of the Offering raised by First Time Users of the Platform plus Nine (9) Percent of the amount of the Offering raised by all other investors."
- "A % of the total Offering Amount (10% for up to $250,000, 9% for more than $250,000 and $500,000 or less); provided only 5% is charged on investors introduced by the issuer."
- "A % of the total Offering Amount (10% for up to $250,000, 9% for more than $250,000 and $500,000 or less, 8% for more than $500,000 and $750,000 or less, and 7% for more than $750,000); provided only 5% is charged on investors introduced by the issuer."

Interesting cases (Done)
- "The issuer will not owe a commission, whether cash or otherwise, to the Intermediary at the conclusion of the Offering."
- "0"

In [297]:
def extract_commission_fee(commission_text):
    '''
    This function uses regex to extract % commission from the amount raised. 
    '''
    commission_text = commission_text.lower()
    
    pattern = re.compile(
        r"""(?xi)                              # 
        (?:                                    # ┌── two main alternatives ──┐
           \d+(?:\.\d+)?                       # │  “6” or “6.5” or “7.0”    │
           (?:\s*[-–]\s*\d+(?:\.\d+)?)?        # │ optional range “–13” → “7 - 13” │
         |                                     # ├───────────────────────────┤
           \(\s*\d+(?:\.\d+)?\s*\)             # │  parenthesized “(3)”      │
        )                                      # └───────────────────────────┘
        \s*                                    # optional whitespace
        (?:%|percent)                          # “%” or “percent”
        """
    ) 
    matches = pattern.findall(commission_text)
    
    # convert matches from string to float
    # if it's a range like "7 - 13 percent" we'll return both numbers [7.0, 13.0]
    nums = []
    for match in matches:
        # strip out everything except digits, dot, and hyphen
        cleaned = re.sub(r'[^\d\.\-]', '', match)
        if '-' in cleaned:
            lo, hi = cleaned.split('-', 1)
            nums.extend([float(lo), float(hi)])
        else:
            nums.append(float(cleaned))
    
    # handle the case when the % commission fee is not disclosed
    if not nums:
        return 0

    # handle multiple % commission fee
    # assume the presence of tiered commission when there are more than three commissions disclosed
    if len(nums) > 3:
        nums_sorted = sorted(nums, reverse=True)
        group = [nums_sorted[0]] # stores tiered % commission
        outliers = []
        for num in nums_sorted[1:]:
            if group[-1] - num <= 2:
                group.append(num)
            else:
                outliers.append(num)
        # compute average of the main group (i.e. tiered commission), then add outliers
        avg_main = sum(group) / len(group)
        final_rate = avg_main + sum(outliers)
    # otherwise, we just average the commissions
    else:
        final_rate = sum(nums) / len(nums)
        
    
    # handle exception, e.g. "2.0% of total offering plus 2.85% of investment"
    exp_patterns = [
        re.findall(r'(\d+(?:\.\d+)?\s*(?:%|percent)) of total offering plus (\d+(?:\.\d+)?\s*(?:%|percent))', commission_text),
        re.findall(r'(\d+(?:\.\d+)?\s*(?:%|percent)) of total offering. (\d+(?:\.\d+)?\s*(?:%|percent)) per investment', commission_text)
    ]
    for exp_pattern in exp_patterns:
        if exp_pattern:
            # convert to float
            first, second = float(re.sub(r'[^\d\.\-]', '', exp_pattern[0][0])), float(re.sub(r'[^\d\.\-]', '', exp_pattern[0][1]))
            final_rate = first + second
    
    return final_rate

In [299]:
# use regex to extract commission from the text
ml_df['RAISEDCOMMISSION'] = np.nan
for idx in ml_df.index:
    ml_df.loc[idx, 'RAISEDCOMMISSION'] = extract_commission_fee(ml_df.loc[idx, 'TEXT_RAISEDCOMMISSION'])

Platform Fee (Equity Stake) Feature

Platform equity stake examples

Platform wants equity stake
- "The Intermediary will also receive compensation in the form of securities equal to two percent (2%) of the total number of the securities sold in the offering."
- "The Intermediary will receive a security-compensation equal to two percent (2%) of the total number of Securities sold in the Offering."
- "Up to 2% or 200 basis points of the success fee may be taken as equity in company based on the same security pursuant to this offering."
- "Securities equal to 2.0% of the securities sold will be issued to the intermediary upon a successful fundraise."
- "SI Securities will receive equity compensation equal to 5% of the number of securities sold."

Platform that does not want equity stake
- "MainVest, Inc. owns no interest in the Company, directly or indirectly, and will not acquire an interest as part of the Offering, nor is there any arrangement for MainVest, Inc. to acquire an interest."
- "The issuer will not owe a cash commission, or any other direct or indirect interest in the issuer, to the intermediary at the conclusion of the offering."
- "None"
- "No"
- nan

Multiple pecentages
- "the intermediary will also receive compensation in the form of securities equal to (i) 2% of the securities sold for any amounts raised up to two million dollars; (ii) 1% of the securities sold for any amounts raised b/w two million & three million dollars"
- "the intermediary will also receive compensation in the form of securities equal to: (i) 2% of the securities sold for any amounts raised between $0.00- $2m; 1% for any amounts raised $2,000,000.01 - $4m; 0.5% for any amounts raised $4,000,000.01 - $5m."
- "securities commission equal to (x) two percent (2%) up to $2,000,000, plus (y) one percent (1%) above $2,000,000 up to $4,000,000, plus (z) one-half percent (0.5%) above $4,000,000."
- "a securities commission of 2% for amounts raised under $2m, 1% between $2m and $4m and .5% for amounts raised between $4m and $5m." 
- "equity safe notes amounting to 2% of the dollar amount raised through the portal. all in 7% (5% commission and 2% in safe)."
- "equity safe notes amounting to 2% of the dollar amount raised through the portal. all in 7% (5% commission and 2% in safe)."
- "2% of raise amount in safe equity. the total fee of 7% (5% cash and 2% equity of total amount raised)."
- "the intermediary will also receive compensation in the form of securities equal to (a) 2% of of any amounts raised up to $2,000,000, (b) 1% of amount raised $2,000,000.01-$4,000,000.00, (c) 0.5% of the amount raised 4,000,000.01-$5,000,000.00"
- "5% commission plus 2% safe equity for a total of 7%"

In [301]:
def extract_equity_fee(commission_text):
    '''
    This function uses regex to extract % platform equity stake from CF offering. 
    '''
    # handle cases when commission_text is np.nan
    if pd.isna(commission_text):
        return 0
    
    commission_text = commission_text.lower()
    
    pattern = re.compile(r'\d+(?:\.\d+)?\s*(?:%|percent)', re.IGNORECASE)
    matches = pattern.findall(commission_text)
    
    # convert matches from string to float
    nums = []
    for match in matches:
        # strip out everything except digits, dot, and hyphen
        cleaned = re.sub(r'[^\d\.]', '', match)
        nums.append(float(cleaned))
    
    # handle the case when the % equity stake fee is not disclosed
    if not nums:
        return 0

    # handle multiple % equity stake fee
    if len(nums) > 1:
        if 5 in nums and 2 in nums: # There are about 10 instances of: "5% commission plus 2% safe equity for a total of 7%"
            return 2
        else:
            return sum(nums) / len(nums)
    # only one % equity stake fee
    else:
        return nums[0]

In [303]:
# use regex to extract equity stake fee from the text
ml_df['EQUITYCOMMISSION'] = np.nan
for idx in ml_df.index:
    ml_df.loc[idx, 'EQUITYCOMMISSION'] = extract_equity_fee(ml_df.loc[idx, 'TEXT_EQUITYCOMMISSION'])

Platform popularity Feature
- The mathematical formula used here is described in "Idea Btainstorm.docx" file
- Even though we are using data points between 2018 and 2023 to train our ML model, in computing platform popularity, we also include data points from 2016 and 2017. This ensures that the platform popularity figure for the 2018 data points are not triveal, i.e. meaningful.

In [ ]:
# # Compute platform popularity
#     # This asusmes that the ml_df has been sorted by the FILING_NUMBER, or equivalently the FILING_DATE
# ml_df['PLATFORMPOPULARITY'] = np.nan
# for i in range(len(ml_df)):
#     # Compute the starting date of the 12-month window
#     end_window = ml_df.loc[ml_df.index[i], 'FILING_DATE']
#     start_Window = end_window - pd.DateOffset(months=12)
#     platform = ml_df.loc[ml_df.index[i], 'COMMISSIONCIK']
    
#     # Filter to only include past CFs with deadline date between FILING_DATE - 12 months and FILING_DATE
#     total_success = 0
#     platform_success = 0
#     for k in range(i):
#         past_deadline = ml_df.loc[ml_df.index[k], 'FINALDEADLINEDATE']
#         if (start_Window <= past_deadline <= end_window) and (ml_df.loc[ml_df.index[k], 'Y'] == 1):
#             # Update the total number of successful past CFs within the 12-month window, in all platforms
#             total_success += 1
            
#             # Update the total number of successful past CFs wihtin the 12-month windoow, in the selected platform
#             if ml_df.loc[ml_df.index[k], 'COMMISSIONCIK'] == platform:
#                 platform_success += 1
                
#     # To prevent error division, i.e. 0 divided by 0
#     if total_success > 0:
#         ml_df.loc[ml_df.index[i], 'PLATFORMPOPULARITY'] = platform_success / total_success
#     else:
#         ml_df.loc[ml_df.index[i], 'PLATFORMPOPULARITY'] = 0

In [305]:
# Note that this code does exactly the same as the above, but this one is just much faster
dates = ml_df['FILING_DATE'].values
deadlines = ml_df['FINALDEADLINEDATE'].values
platforms = ml_df['COMMISSIONCIK'].values
success = ml_df['Y'].values.astype(bool)

pop_scores = []

for i, (d, p) in enumerate(zip(dates, platforms)):
    start = d - np.timedelta64(12, 'M')  # equivalent to pd.DateOffset
    window_mask = (deadlines[:i] >= start) & (deadlines[:i] <= d) & success[:i]
    total = window_mask.sum()
    if total == 0:
        pop_scores.append(0)
    else:
        pop_scores.append(((platforms[:i] == p) & window_mask).sum() / total)
    
ml_df['PLATFORMPOPULARITY'] = pop_scores

Offering Duration Feature

In [308]:
# Compute offering duration
ml_df['DURATION'] = ml_df['FINALDEADLINEDATE'] - ml_df['FILING_DATE']

# Change data type from datetime to float
ml_df['DURATION'] = ml_df['DURATION'].dt.days.astype(float)

Number of Competitors Feature

In [ ]:
# # To store the number of CF competitors
# ml_df['COMPETITORSCOUNT'] = np.nan

# # Compute the number of CF competitors
# # Note that that ml_df dataset has been sorted by FILE_NUMBER (which also means that it has been sorted by FILING_DATE)
# for i in range(len(ml_df)):
#     curr_start_date = ml_df.iloc[i,:]['FILING_DATE']
#     count_competitions = 0
    
#     # count the number of comptitors with FILING_DATE before the currently selected CF
#     for k in range(i):
#         comp_deadline_date = ml_df.iloc[k,:]['FINALDEADLINEDATE']
#         if comp_deadline_date >= curr_start_date:
#             count_competitions += 1
    
#     # count the number of competitors with the same FILING_DATE as the currently selected CF
#     add = 1
#     while i + add < len(ml_df) and ml_df.iloc[i + add,:]['FILING_DATE'] == curr_start_date:
#         count_competitions += 1
#         add += 1
    
#     ml_df.loc[ml_df.index[i], 'COMPETITORSCOUNT'] = count_competitions
    
#     if i == 100:
#         break

In [310]:
# IMPORTANT NOTE: This cose does the exact thing as the above, the only difference is the time complexity, which has been improved
# ... from O(n^2) to O(n log(n))
import heapq

# assume ml_df is already sorted by 'FILING_DATE'
ml_df_temp = ml_df.copy()

# 1) prior_active_count via min‐heap of deadlines
deadlines = []          # min‐heap of previous deadlines
prior_active = []       # to store counts

for start, deadline in zip(ml_df_temp['FILING_DATE'], ml_df_temp['FINALDEADLINEDATE']):
    # pop any deadlines that have already "closed" before this start
    while deadlines and deadlines[0] < start:
        heapq.heappop(deadlines)
    # now len(deadlines) is the number of previous intervals still open
    prior_active.append(len(deadlines))
    # push this offering's deadline for future starts
    heapq.heappush(deadlines, deadline)

ml_df_temp['PRIOR_ACTIVE'] = prior_active

# 2) count how many same‐date peers come *after* each row
#    group_size = total per date, cum_idx = 0,1,2… in group
group_size = ml_df_temp.groupby('FILING_DATE')['FILING_DATE'].transform('size')
cum_idx    = ml_df_temp.groupby('FILING_DATE').cumcount()
peers_after = group_size - cum_idx - 1

ml_df_temp['PEERS_AFTER'] = peers_after

# 3) competitor count = prior_active + peers_after
ml_df_temp['COMPETITORSCOUNT'] = ml_df_temp['PRIOR_ACTIVE'] +ml_df_temp['PEERS_AFTER']

# cleanup
ml_df_temp = ml_df_temp.drop(columns=['PRIOR_ACTIVE','PEERS_AFTER'])

ml_df['COMPETITORSCOUNT'] = ml_df_temp['COMPETITORSCOUNT']

Type of Security Offered Feature: Equity, Debt, SAFE, or Other

In [312]:
# Renaming the categories
ml_df['SECURITYOFFEREDTYPE'] = ml_df['SECURITYOFFEREDTYPE'].map({
    'Other': 'Complex',
    'Common Stock': 'Equity',
    'Debt': 'Debt',
    'Preferred Stock': 'Equity'
})

In [314]:
def extract_security_offered(text):
    '''
    This function uses regex to classify the type of secutiy offered in a CF offering.
    The function checks for patterns in a specific order: SAFE > Debt > Equity > Other.
    '''
    # Special case: When text is np.nan it means that the security type has been disclosed in the SECURITYOFFEREDTYPE column
    if pd.isna(text):
        return np.nan
    
    txt = text.lower()
    
    # 1) SAFE
    # matches 'simple agreement(s) for future equity', 'safe', 'safe+rev'
    if re.search(r'simple agreement(?:s)? for future equity', txt) or re.search(r'\bsafe\b', txt):
        return 'SAFE'
    
    # 2) Debt
    # matches DPA, DPAs, or DUSTO
    if re.search(r'\bdpa(?:s)?\b', txt) or re.search(r'\bdusto\b', txt):
        return 'Debt'
    
    # 3) Equity
    # check multi-word patterns first
    equity_patterns = [
        r'\bcommon unit(?:s)?\b',
        r'\bllc unit(?:s)?\b',
        r'\bmember unit(?:s)?\b',
        r'\bmembership interest(?:s)?\b',
        r'\bpartnership equity\b',
        r'\bpreferred\b',
        r'\bclass\b',
        r'\bseries\b',
        r'\bcommon\b'
    ]
    for pat in equity_patterns:
        if re.search(pat, txt):
            return 'Equity'
    
    # 4) Others
    return 'Other'

In [316]:
# extract security type from SECURITYOFFEREDOTHERDESC
ml_df['EXTRACTED_SECURITYOFFEREDOTHERDESC'] = np.nan
for i in range(len(ml_df)):
    text = ml_df.iloc[i,:]['SECURITYOFFEREDOTHERDESC']
    ml_df.loc[ml_df.index[i], 'EXTRACTED_SECURITYOFFEREDOTHERDESC'] = extract_security_offered(text)

In [318]:
# combine the results from SECURITYOFFEREDTYPE and EXTRACTED_SECURITYOFFEREDOTHERDESC
ml_df['FINALSECURITYOFFEREDTYPE'] = np.nan
for i in range(len(ml_df)):
    if ml_df.iloc[i,:]['SECURITYOFFEREDTYPE'] == 'Complex':
        ml_df.loc[ml_df.index[i], 'FINALSECURITYOFFEREDTYPE'] = ml_df.iloc[i,:]['EXTRACTED_SECURITYOFFEREDOTHERDESC']
    else:
        ml_df.loc[ml_df.index[i], 'FINALSECURITYOFFEREDTYPE'] = ml_df.iloc[i,:]['SECURITYOFFEREDTYPE']

Price Determination Method Feature

In [320]:
def extract_pricing_method(text, sec_type):
    """
    Return 0 if pricing is ARBITRARY, 1 if NON-ARBITRARY.
    
    text: str or NaN
    sec_type: 'SAFE', 'Equity', etc.
    """
    # 0) methodology not specified
    if pd.isna(text):
        return 0
    
    # 1) normalize
    t = text.lower().strip()
    
    # 2) obvious arbitrary keywords
    arb_kw = [
        r'\barbitrar(?:y|ity)\b',
        r'\bdiscretion\b',
        r'to be determined'
    ]
    for p in arb_kw:
        if re.search(p, t):
            return 0
    
    # 3) SAFE‐specific arbitrary patterns
    if sec_type.upper() == 'SAFE':
        safe_patterns = [
            r'price will be set upon conversion after a qualifying event in the future',
            r'^\d+(?:,\d{3})*(?:\.\d+)?$',                      # only a number like "1.0", no other text description
            r'\$\s*\d+(?:,\d{3})*(?:\.\d+)?\s*per '
        ]
        for p in safe_patterns:
            if re.search(p, t):
                return 0
    
    # 4) Equity‐specific arbitrary patterns
    if sec_type.lower() == 'equity':
        eq_patterns = [
            r'\bmanagement\b',                              # management determination
            r'\bmanaging\b',
            r'\$\s*\d+(?:,\d{3})*(?:\.\d+)?\s*per ',        # $1.00 per share / unit
            r'based on target number of securities',        # formulaic but non-traditional
            r'good faith'                                   # good faith determination
        ]
        for p in eq_patterns:
            if re.search(p, t):
                return 0
    
    # 5) If none of the above matched, assume NON-ARBITRARY
    return 1

In [322]:
# Extract on whether pricing methodology was arbitrary or not, using regex
ml_df['IS_PRICEMETHOD'] = np.nan
for idx in ml_df.index:
    ml_df.loc[idx, 'IS_PRICEMETHOD'] = extract_pricing_method(ml_df.loc[idx, 'PRICEDETERMINATIONMETHOD'], ml_df.loc[idx, 'FINALSECURITYOFFEREDTYPE'])

### Issuer Characteristics Features

Company Age Feature

In [324]:
# Compaute the company age
ml_df['COMPANYAGE'] = (ml_df['FILING_DATE'] - ml_df['DATEINCORPORATION']).dt.days

Number of Past Successful CF (By The Same Issuer) Feature

In [326]:
# Compute the number of past successful CF, by the same issuer
# Step 1: Sort the entire DataFrame first by 'CIK' and then by 'FILING_DATE'
ml_df_sorted = ml_df.sort_values(by=['CIK', 'FILING_DATE'], ascending=[True, True])

# Step 2: Group by 'CIK' and compute the cumulative sum of 'Y'
ml_df_sorted['CUMULATIVE_Y'] = ml_df_sorted.groupby('CIK')['Y'].cumsum()

# Step 3: Subtract the current row’s Y to leave only *prior* successes
ml_df_sorted['PASTCOMPANYSUCCESS'] = ml_df_sorted['CUMULATIVE_Y'] - ml_df_sorted['Y']

# Step 4: Transfer the PASTCOMPANYSUCCESS from ml_df_sorted to ml_df
ml_df['PASTCOMPANYSUCCESS'] = np.nan
for i in range(len(ml_df_sorted)):
    ml_df.loc[ml_df_sorted.index[i], 'PASTCOMPANYSUCCESS'] = ml_df_sorted.iloc[i,:]['PASTCOMPANYSUCCESS']

Number of Past Failed CF (By The Same Issuer) Feature

In [328]:
# Compute the number of past failed CF, by the same issuer
# Step 1.1: Sort the entire DataFrame first by 'CIK' and then by 'FILING_DATE'
ml_df_sorted = ml_df.sort_values(by=['CIK', 'FILING_DATE'], ascending=[True, True])

# Step 1.2: Compute the reverse of Y, i.e. 0 becomes 1 and 1 becomes 0
ml_df_sorted['REVERSE_Y'] = ml_df_sorted['Y'].map({0:1, 1:0})

# Step 2: Group by 'CIK' and compute the cumulative sum of 'REVERSE_Y'
ml_df_sorted['CUMULATIVE_REVERSE_Y'] = ml_df_sorted.groupby('CIK')['REVERSE_Y'].cumsum()

# Step 3: Subtract the current row’s REVERSE_Y to leave only *prior* failures
ml_df_sorted['PASTCOMPANYFAILURE'] = ml_df_sorted['CUMULATIVE_REVERSE_Y'] - ml_df_sorted['REVERSE_Y']

# Step 4: Transfer the PASTCOMPANYFAILURE from ml_df_sorted to ml_df
ml_df['PASTCOMPANYFAILURE'] = np.nan
for i in range(len(ml_df_sorted)):
    ml_df.loc[ml_df_sorted.index[i], 'PASTCOMPANYFAILURE'] = ml_df_sorted.iloc[i,:]['PASTCOMPANYFAILURE']

Number of Past Successful CF (By The Issuing Person) Feature

In [330]:
# Clean up the issuing person's name
    # We need to take car of the following.
        # There is this weired tag, e.g. "/s/ Adam Berry" and "/c/ Charles Potter"
        # There are also cases where, e.g. "/Gregory Miller/"
        # There are also cases where there are multiple white-spces, e.g. "Bruce  Goldblatt"
        # Remove title, e.g. 'John Terry, PhD', 'Johnathan Aho, M.D., PhD', 'Dr.' etc.

ml_df['ISSUERSIGNATURE'] = (
    ml_df['ISSUERSIGNATURE']
      # 0) lowercase everything
      .str.lower()
      # 1) remove /s/, /c/, any single-letter tags and stray slashes
      .str.replace(r'/[a-z]/\s*|/', '', regex=True)
      # 2) strip leading "dr."
      .str.replace(r'^\s*dr\.?\s+', '', regex=True)
      # 3) strip trailing ", phd", ", m.d.", ", m.d., phd", etc.
      .str.replace(
          r',\s*(?:m\.?d\.?|ph\.?d\.?)(?:\s*,\s*(?:m\.?d\.?|ph\.?d\.?))*\s*$',
          '',
          regex=True
      )
      # 4) collapse multiple white-spaces into one
      .str.replace(r'\s+', ' ', regex=True)
      # 5) remove trailing and the opposite of trailing white-spaces
      .str.strip()
)

In [332]:
# Compute the number of past successful CF, by the same issuing person
# Step 1: Sort the entire DataFrame first by 'ISSUERSIGNATURE' and then by 'FILING_DATE'
ml_df_sorted = ml_df.sort_values(by=['ISSUERSIGNATURE', 'FILING_DATE'], ascending=[True, True])

# Step 2: Group by 'CIK' and compute the cumulative sum of 'Y'
ml_df_sorted['CUMULATIVE_Y'] = ml_df_sorted.groupby('ISSUERSIGNATURE')['Y'].cumsum()

# Step 3: Subtract the current row’s Y to leave only *prior* successes
ml_df_sorted['PASTPERSONSUCCESS'] = ml_df_sorted['CUMULATIVE_Y'] - ml_df_sorted['Y']

# Step 4: Transfer the PASTPERSONSUCCESS from ml_df_sorted to ml_df
ml_df['PASTPERSONSUCCESS'] = np.nan
for i in range(len(ml_df_sorted)):
    ml_df.loc[ml_df_sorted.index[i], 'PASTPERSONSUCCESS'] = ml_df_sorted.iloc[i,:]['PASTPERSONSUCCESS']

Number of Past Failed CF (By The Issuing Person) Feature

In [334]:
# Compute the number of past failed CF, by the same issuer
# Step 1.1: Sort the entire DataFrame first by 'ISSUERSIGNATURE' and then by 'FILING_DATE'
ml_df_sorted = ml_df.sort_values(by=['ISSUERSIGNATURE', 'FILING_DATE'], ascending=[True, True])

# Step 1.2: Compute the reverse of Y, i.e. 0 becomes 1 and 1 becomes 0
ml_df_sorted['REVERSE_Y'] = ml_df_sorted['Y'].map({0:1, 1:0})

# Step 2: Group by 'ISSUERSIGNATURE' and compute the cumulative sum of 'REVERSE_Y'
ml_df_sorted['CUMULATIVE_REVERSE_Y'] = ml_df_sorted.groupby('ISSUERSIGNATURE')['REVERSE_Y'].cumsum()

# Step 3: Subtract the current row’s REVERSE_Y to leave only *prior* failures
ml_df_sorted['PASTPERSONFAILURE'] = ml_df_sorted['CUMULATIVE_REVERSE_Y'] - ml_df_sorted['REVERSE_Y']

# Step 4: Transfer the PASTCOMPANYFAILURE from ml_df_sorted to ml_df
ml_df['PASTPERSONFAILURE'] = np.nan
for i in range(len(ml_df_sorted)):
    ml_df.loc[ml_df_sorted.index[i], 'PASTPERSONFAILURE'] = ml_df_sorted.iloc[i,:]['PASTPERSONFAILURE']

Incorporated in Delaware Boolean Feature

In [336]:
# Obtain whether the company was registered in Delaware (DE) or not
ml_df['ISDELAWARE'] = np.nan
for idx in ml_df.index:
    if ml_df.loc[idx, 'JURISDICTIONORGANIZATION'] == 'DE':
        ml_df.loc[idx, 'ISDELAWARE'] = 1
    else:
        ml_df.loc[idx, 'ISDELAWARE'] = 0

### Geographic Characteristics Features

Local Competition Feature

In [338]:
# Apply filter to only include companies that are operating in the USA
us_states = {
    'AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','IA','ID','IL','IN',
    'KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH',
    'NJ','NM','NV','NY','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VA',
    'VT','WA','WI','WV','WY'
}

mask = ml_df['STATEORCOUNTRY'].isin(us_states)
ml_df = ml_df.loc[mask, :].copy()

In [ ]:
# # To store the number of local CF competitors
# ml_df['LOCALCOMPETITORSCOUNT'] = np.nan

# # Compute the number of local CF competitors
# # Note that that ml_df dataset has been sorted by FILE_NUMBER (which also means that it has been sorted by FILING_DATE)
# for i in range(len(ml_df)):
#     curr_start_date = ml_df.iloc[i,:]['FILING_DATE']
#     curr_state = ml_df.iloc[i,:]['STATEORCOUNTRY']
#     count_competitions = 0
    
#     # count the number of comptitors in the same state with FILING_DATE before the currently selected CF
#     for k in range(i):
#         comp_deadline_date = ml_df.iloc[k,:]['FINALDEADLINEDATE']
#         comp_state = ml_df.iloc[k,:]['STATEORCOUNTRY']
#         if comp_deadline_date >= curr_start_date and comp_state == curr_state:
#             count_competitions += 1
    
#     # count the number of competitors with the same FILING_DATE as the currently selected CF
#     add = 1
#     while i + add < len(ml_df) and ml_df.iloc[i + add,:]['FILING_DATE'] == curr_start_date and ml_df.iloc[i + add,:]['STATEORCOUNTRY'] == curr_state:
#         count_competitions += 1
#         add += 1
    
#     ml_df.loc[ml_df.index[i], 'LOCALCOMPETITORSCOUNT'] = count_competitions
    
#     if i == 200:
#         break

In [340]:
# This is the exact same as the above, but it is a lot faster

# Prepare a heap per state
deadlines_by_state = {state: [] for state in ml_df['STATEORCOUNTRY'].unique()}

# Compute local competitor counts
local_counts = []
for start, deadline, state in zip(ml_df['FILING_DATE'],
                                 ml_df['FINALDEADLINEDATE'],
                                 ml_df['STATEORCOUNTRY']):
    heap = deadlines_by_state[state]
    # Remove any deadlines that ended before this start date
    while heap and heap[0] < start:
        heapq.heappop(heap)
    # Now heap contains only past CFs in this state still “active”
    local_counts.append(len(heap))
    # Push this CF’s deadline for future rows
    heapq.heappush(heap, deadline)

ml_df['LOCALCOMPETITORSCOUNT'] = local_counts

Creative Workforce Feature

In [342]:
# Process workforce dataset
creative_df = pd.DataFrame({'YEAR':[], 'STATE':[], 'PROPORTION1000':[]})
for i in range(16,25):
    folder_name = f'oesm{i}st'
    file_name = f'state_M20{i}_dl.xlsx'

    # import the excel file
    file_path = f'demographic_dataset/workforce_dataset/{folder_name}/{file_name}'
    workforce_df = pd.read_excel(file_path)
    
    # extract the proportion per 1000 of people working in creative industry in each state
    if 'ST' in workforce_df.columns: # Between 2016 and 2019, state is stored in 'ST' column
        for state in workforce_df['ST'].unique():
            mask = (workforce_df['ST'] == state) & (workforce_df['OCC_TITLE'] == 'Arts, Design, Entertainment, Sports, and Media Occupations')
            creative_proportion = workforce_df.loc[mask, 'JOBS_1000'].iloc[0]
            creative_df.loc[len(creative_df)] = [i + 2000, state, creative_proportion]
    elif 'PRIM_STATE' in workforce_df.columns: # Between 2020 and 2025, state is stored in 'PRIM_STATE' column
        for state in workforce_df['PRIM_STATE'].unique():
            mask = (workforce_df['PRIM_STATE'] == state) & (workforce_df['OCC_TITLE'] == 'Arts, Design, Entertainment, Sports, and Media Occupations')
            creative_proportion = workforce_df.loc[mask, 'JOBS_1000'].iloc[0]
            creative_df.loc[len(creative_df)] = [i + 2000, state, creative_proportion]

In [343]:
# Add the proportion of creative workforce to ml_df
ml_df['CREATIVEWORKFORCE'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    curr_year = ml_df.loc[idx, 'FILING_DATE'].year
    mask = (creative_df['STATE'] == curr_state) & (creative_df['YEAR'] == curr_year)
    ml_df.loc[idx, 'CREATIVEWORKFORCE'] = creative_df.loc[mask, 'PROPORTION1000'].iloc[0]

RUCC (Rural-Urban Continuum Code) Feature
- This measure the centrality / peripherility of US states

In [345]:
# import RUCC data
file_path = 'demographic_dataset/rucc_dataset/Ruralurbancontinuumcodes2023.xlsx'
rucc_df = pd.read_excel(file_path)

In [346]:
# 1) Compute total population and population-weighted RUCC sum per state
pop_sum = rucc_df.groupby('State')['Population_2020'].sum()
rucc_pop = (rucc_df['RUCC'] * rucc_df['Population_2020']).groupby(rucc_df['State']).sum()

# 2) Weighted average RUCC per state
state_rucc = (rucc_pop / pop_sum).rename('RUCC')

# 3) Map into ml_df
ml_df['RUCC'] = ml_df['STATEORCOUNTRY'].map(state_rucc)

Income Level Feature

In [348]:
# import personal income per capita
file_path = 'demographic_dataset/income_dataset/income_dataset.xlsx'
income_df = pd.read_excel(file_path)

In [349]:
# Add the income level to ml_df
ml_df['INCOMELEVEL'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    curr_year = ml_df.loc[idx, 'FILING_DATE'].year
    mask = (income_df['STATE_CODE'] == curr_state)
    ml_df.loc[idx, 'INCOMELEVEL'] = income_df.loc[mask, curr_year].iloc[0]

Age Feature

In [351]:
# import median age dataset
file_path = 'demographic_dataset/age_dataset/ACSST1Y2023.xlsx'
age_df = pd.read_excel(file_path)

# Process the age_df
age_df = age_df.iloc[:, [i for i in range(1, len(age_df.columns), 12)]]
new_cols = []
for col in age_df.columns:
    col = col.lower()
    match = re.findall(r'([a-z\s]+).*', col)[0]
    new_cols.append(match)
age_df.columns = new_cols

In [352]:
# 2) map of lower‐case state names → postal codes
state_name_to_code = {
    'alabama':              'AL',
    'alaska':               'AK',
    'arizona':              'AZ',
    'arkansas':             'AR',
    'california':           'CA',
    'colorado':             'CO',
    'connecticut':          'CT',
    'delaware':             'DE',
    'district of columbia': 'DC',
    'florida':              'FL',
    'georgia':              'GA',
    'hawaii':               'HI',
    'idaho':                'ID',
    'illinois':             'IL',
    'indiana':              'IN',
    'iowa':                 'IA',
    'kansas':               'KS',
    'kentucky':             'KY',
    'louisiana':            'LA',
    'maine':                'ME',
    'maryland':             'MD',
    'massachusetts':        'MA',
    'michigan':             'MI',
    'minnesota':            'MN',
    'mississippi':          'MS',
    'missouri':             'MO',
    'montana':              'MT',
    'nebraska':             'NE',
    'nevada':               'NV',
    'new hampshire':        'NH',
    'new jersey':           'NJ',
    'new mexico':           'NM',
    'new york':             'NY',
    'north carolina':       'NC',
    'north dakota':         'ND',
    'ohio':                 'OH',
    'oklahoma':             'OK',
    'oregon':               'OR',
    'pennsylvania':         'PA',
    'rhode island':         'RI',
    'south carolina':       'SC',
    'south dakota':         'SD',
    'tennessee':            'TN',
    'texas':                'TX',
    'utah':                 'UT',
    'vermont':              'VT',
    'virginia':             'VA',
    'washington':           'WA',
    'west virginia':        'WV',
    'wisconsin':            'WI',
    'wyoming':              'WY',
    'puerto rico':          'PR'
}

# 3) rename in place
age_df.rename(columns=state_name_to_code, inplace=True)

In [353]:
# Add the age median to ml_df
ml_df['AGE'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    ml_df.loc[idx, 'AGE'] = age_df.loc[0, curr_state]

Education Feature
- The proportion of people aged 25 or older with at least a Bachelor's degree

In [354]:
# import education dataset
file_path = 'demographic_dataset/education_dataset/education_dataset.xlsx'
education_df = pd.read_excel(file_path)

In [355]:
# Add the education feature to ml_df
ml_df['BACHELOREDUCATION'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    curr_year = ml_df.loc[idx, 'FILING_DATE'].year
    mask = (education_df['YEAR'] == curr_year)
    ml_df.loc[idx, 'BACHELOREDUCATION'] = education_df.loc[mask, curr_state].iloc[0]

In [356]:
ml_df_geochar = ml_df.copy()

### Macro-Economy Features

Unemployment Rate Feature

In [357]:
# import unemployment rate
file_path = 'macroeconomy_dataset/unemployment_dataset/unemployment_dataset.xlsx'
unemployment_df = pd.read_excel(file_path)

In [358]:
# Add the unemployment rate feature to ml_df
ml_df['UNEMPLOYMENTRATE'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    curr_year = ml_df.loc[idx, 'FILING_DATE'].year
    mask = (unemployment_df['Year'] == curr_year)
    ml_df.loc[idx, 'UNEMPLOYMENTRATE'] = unemployment_df.loc[mask, curr_state].iloc[0]

Spot Interest Rate Feature

In [360]:
# Import 10-year US governmnet bond yield
file_path = 'macroeconomy_dataset/interestrate_dataset/interestrate_dataset.xlsx'
interestrate_df = pd.read_excel(file_path)

In [361]:
# use forward-fill to fill up interest rate during the weekend
interestrate_df.set_index('observation_date', inplace=True)
full_date = pd.date_range(start=interestrate_df.index.min(),
                         end=interestrate_df.index.max(),
                         freq='D')
interestrate_df = interestrate_df.reindex(full_date).ffill()

In [362]:
# Add the spot interest rate feature to ml_df
ml_df['SPOTINTERESTRATE'] = np.nan
for idx in ml_df.index:
    curr_date = ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)
    ml_df.loc[idx, 'SPOTINTERESTRATE'] = interestrate_df.loc[curr_date, 'DGS10']

Short-Term Average Interest Rate Feature
- Compute 30-day average, up to the start offering date

In [364]:
# Import 10-year US governmnet bond yield
file_path = 'macroeconomy_dataset/interestrate_dataset/interestrate_dataset.xlsx'
interestrate_df = pd.read_excel(file_path)

In [365]:
# Compute the 30-day average
interestrate_df['AVG'] = interestrate_df['DGS10'].rolling(window=30, min_periods=1).mean()

# use forward-fill to fill up average interest rate during the weekend
interestrate_df.set_index('observation_date', inplace=True)
full_date = pd.date_range(start=interestrate_df.index.min(),
                         end=interestrate_df.index.max(),
                         freq='D')
interestrate_df = interestrate_df.reindex(full_date).ffill()

In [366]:
# Add the average interest rate feature to ml_df
ml_df['AVERAGEINTERESTRATE'] = np.nan
for idx in ml_df.index:
    curr_date = ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)
    ml_df.loc[idx, 'AVERAGEINTERESTRATE'] = interestrate_df.loc[curr_date, 'AVG']

Inflation Rate Feature

In [367]:
# Compute the 12-month change in CPI index to approximate inflation rate
combined_cpi_df = pd.DataFrame({'Region':[], 'Year':[], 'Month':[], 'INFLATIONRATE':[]})
census_regions = ['midwest', 'northeast', 'south', 'west']
for region in census_regions:
    # Import the cpi index
    file_path = f'macroeconomy_dataset/cpi_dataset/cpi_{region}.xlsx'
    cpi_df = pd.read_excel(file_path)
    cpi_df.set_index('Year', inplace=True)
    cpi_df.drop(['Annual', 'HALF1', 'HALF2'], axis=1, inplace=True)
    
    # Unpivot the dataset
    cpi_df = cpi_df.stack() # unpivot the dataset
    cpi_df = cpi_df.reset_index().rename(columns={'level_1':'Month', 0:'CPI'})
    
    # Compute the 12-month change in CPI index to approximate inflation rate
    cpi_df['INFLATIONRATE'] = cpi_df['CPI'].pct_change(periods=12) * 100
    
    # Combine the dataset
    cpi_df['Region'] = region
    combined_cpi_df = pd.concat(
        [combined_cpi_df, cpi_df[['Region','Year','Month','INFLATIONRATE']]],
        ignore_index=True
    )

# Rename the month, e.g. Jan -> 1, Feb -> 2
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sep': 9, 'Oct': 10,'Nov': 11,'Dec': 12
}
combined_cpi_df['Month'] = combined_cpi_df['Month'].map(month_map)

In [368]:
# Compute the inflation rate feature to ml_df
ml_df['INFLATIONRATE'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    curr_year = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).year
    curr_month = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).month
    
    # create mapping for State -> Census Region
    state_to_region = {
        # Northeast
        **dict.fromkeys(['CT','ME','MA','NH','RI','VT','NJ','NY','PA'], 'northeast'),
        # Midwest
        **dict.fromkeys(['IL','IN','MI','OH','WI','IA','KS','MN','MO','NE','ND','SD'], 'midwest'),
        # South
        **dict.fromkeys([
            'DE','DC','FL','GA','MD','NC','SC','VA','WV',
            'AL','KY','MS','TN','AR','LA','OK','TX'
        ], 'south'),
        # West
        **dict.fromkeys([
            'AZ','CO','ID','MT','NV','NM','UT','WY',
            'AK','CA','HI','OR','WA'
        ], 'west')
    }
    curr_region = state_to_region[curr_state]
    
    mask = (combined_cpi_df['Year'] == curr_year) & (combined_cpi_df['Month'] == curr_month) & (combined_cpi_df['Region'] == curr_region)
    ml_df.loc[idx, 'INFLATIONRATE'] = combined_cpi_df.loc[mask, 'INFLATIONRATE'].iloc[0]

Economic Policy Uncertainty (EPU) Feature

In [369]:
# Import EPU dataset
file_path = 'macroeconomy_dataset/epu_dataset/epu_dataset.xlsx'
epu_df = pd.read_excel(file_path)

In [370]:
# Only keep the EPU_Composite
for col in epu_df.columns:
    if re.findall(f'EPU_Composite..', col) or col == 'year' or col == 'month':
        pass
    else:
        epu_df.drop(col, axis=1, inplace=True)
        
# Rename the columns
new_cols = ['year', 'month']
epu_df.columns = new_cols + [col[-2:] for col in epu_df.columns[2:]]

In [371]:
# Compute the EPU into ml_df
ml_df['EPU'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    curr_year = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).year
    curr_month = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).month
    mask = (epu_df['year'] == curr_year) & (epu_df['month'] == curr_month)
    ml_df.loc[idx, 'EPU'] = epu_df.loc[mask, curr_state].iloc[0]

Consumer Sentiment Feature

In [372]:
# Import Consumner Sentiment index dataset
file_path = 'macroeconomy_dataset/sentiment_dataset/sentiment_dataset.xlsx'
sentiment_df = pd.read_excel(file_path)

In [373]:
# Compute the Consumer Sentiment index into ml_df
ml_df['CONSUMERSENTIMENT'] = np.nan
for idx in ml_df.index:
    curr_year = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).year
    curr_month = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).month
    mask = (sentiment_df['Year'] == curr_year) & (sentiment_df['Month'] == curr_month)
    ml_df.loc[idx, 'CONSUMERSENTIMENT'] = sentiment_df.loc[mask, 'Index'].iloc[0]

Expected Change in Unemployment Feature

In [374]:
# Import Expected Change in Unemployment dataset
file_path = 'macroeconomy_dataset/expected_change_unemployment/expected_change_unemployment.xlsx'
exp_unemployment_df = pd.read_excel(file_path)

In [375]:
# Compute the CExpected Change in Unemployment dataset into ml_df
ml_df['EX_UNEMPLOYMENTRATE'] = np.nan
for idx in ml_df.index:
    curr_year = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).year
    curr_month = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).month
    mask = (exp_unemployment_df['Year'] == curr_year) & (exp_unemployment_df['Month'] == curr_month)
    ml_df.loc[idx, 'EX_UNEMPLOYMENTRATE'] = exp_unemployment_df.loc[mask, 'Relative'].iloc[0]

Expected Change in Interest Rates Feature

In [376]:
# Import Expected Change in Interest Rate dataset
file_path = 'macroeconomy_dataset/expected_change_interest/expected_change_interest.xlsx'
exp_interest_df = pd.read_excel(file_path)

In [377]:
# Compute the Expected Change in Interest Rate into ml_df
ml_df['EX_SPOTINTERESTRATE'] = np.nan
for idx in ml_df.index:
    curr_year = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).year
    curr_month = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).month
    mask = (exp_interest_df['Year'] == curr_year) & (exp_interest_df['Month'] == curr_month)
    ml_df.loc[idx, 'EX_SPOTINTERESTRATE'] = exp_interest_df.loc[mask, 'Relative'].iloc[0]

Expected Change in Prices (Inflation) Feature

In [378]:
# Import Expected Change in Price dataset
file_path = 'macroeconomy_dataset/expected_change_price/expected_change_price.xlsx'
exp_inflation_df = pd.read_excel(file_path)

In [379]:
# Compute the Expected Change in Price into ml_df
ml_df['EX_INFLATIONRATE'] = np.nan
for idx in ml_df.index:
    curr_year = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).year
    curr_month = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).month
    mask = (exp_inflation_df['Year'] == curr_year) & (exp_inflation_df['Month'] == curr_month)
    ml_df.loc[idx, 'EX_INFLATIONRATE'] = exp_inflation_df.loc[mask, 'Mean'].iloc[0]

Gini Index Feature

In [380]:
# Import Gini Index dataset
file_path = 'macroeconomy_dataset/gini_dataset/gini_dataset.xlsx'
gini_df = pd.read_excel(file_path)

# Add state code column
state_name_to_code = {
    'united states':        'US',
    'alabama':              'AL',
    'alaska':               'AK',
    'arizona':              'AZ',
    'arkansas':             'AR',
    'california':           'CA',
    'colorado':             'CO',
    'connecticut':          'CT',
    'delaware':             'DE',
    'district of columbia': 'DC',
    'florida':              'FL',
    'georgia':              'GA',
    'hawaii':               'HI',
    'idaho':                'ID',
    'illinois':             'IL',
    'indiana':              'IN',
    'iowa':                 'IA',
    'kansas':               'KS',
    'kentucky':             'KY',
    'louisiana':            'LA',
    'maine':                'ME',
    'maryland':             'MD',
    'massachusetts':        'MA',
    'michigan':             'MI',
    'minnesota':            'MN',
    'mississippi':          'MS',
    'missouri':             'MO',
    'montana':              'MT',
    'nebraska':             'NE',
    'nevada':               'NV',
    'new hampshire':        'NH',
    'new jersey':           'NJ',
    'new mexico':           'NM',
    'new york':             'NY',
    'north carolina':       'NC',
    'north dakota':         'ND',
    'ohio':                 'OH',
    'oklahoma':             'OK',
    'oregon':               'OR',
    'pennsylvania':         'PA',
    'rhode island':         'RI',
    'south carolina':       'SC',
    'south dakota':         'SD',
    'tennessee':            'TN',
    'texas':                'TX',
    'utah':                 'UT',
    'vermont':              'VT',
    'virginia':             'VA',
    'washington':           'WA',
    'west virginia':        'WV',
    'wisconsin':            'WI',
    'wyoming':              'WY',
    'puerto rico':          'PR'
}
gini_df['State'] = gini_df['Location'].str.lower().map(state_name_to_code)

In [381]:
# There is an absent on data point for the year 2020
# 1) Build a full MultiIndex of every Location × every Year you care about
locations = gini_df['Location'].unique()
years     = np.arange(2016, 2024)            # adjust to your overall span
full_index = pd.MultiIndex.from_product(
    [locations, years],
    names=['Location','TimeFrame']
)

# 2) Reindex your DataFrame onto that full grid (this will insert a 2020 row per Location, with Gini = NaN)
gini_df = (
    gini_df
      .set_index(['Location','TimeFrame'])         # pivot into the MultiIndex
      .reindex(full_index)                    # insert missing years
)

# 3) Now interpolate within each Location
gini_df['Gini_imputed'] = (
    gini_df
      .groupby(level='Location')['Data']
      .apply(lambda s: s.interpolate(method='linear'))
)

# 4) Perform forward fill on the rest of the columns
gini_df.ffill(inplace=True)

# bring Year & Location back as columns
gini_df = gini_df.reset_index()

# Dtop the 'Data' column
gini_df.drop('Data', inplace=True, axis=1)

In [382]:
# Compute the Gini Index into ml_df
ml_df['GINI'] = np.nan
for idx in ml_df.index:
    curr_year = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).year
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    mask = (gini_df['TimeFrame'] == curr_year) & (gini_df['State'] == curr_state)
    ml_df.loc[idx, 'GINI'] = gini_df.loc[mask, 'Gini_imputed'].iloc[0]

Geopolitical Risk (GPR) Feature

In [384]:
# Import GPR dataset
file_path = 'macroeconomy_dataset/geopolitical_risk_dataset/data_gpr_export.xls'
gpr_df = pd.read_excel(file_path)

# Only include relevant columns
gpr_df = gpr_df.loc[:, ['month', 'GPRHC_USA']]

In [385]:
# Compute the GPR into ml_df
ml_df['GPR'] = np.nan
for idx in ml_df.index:
    curr_year = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).year
    curr_month = (ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)).month
    mask = (gpr_df['month'].dt.year == curr_year) & (gpr_df['month'].dt.month == curr_month)
    ml_df.loc[idx, 'GPR'] = gpr_df.loc[mask, 'GPRHC_USA'].iloc[0]

VIX Index and S&P500 Feature

In [409]:
import yfinance as yf

In [410]:
# Import s&p and vix dataset
vix_snp_df = pd.read_csv('macroeconomy_dataset/s&p_vix_dataset/s&p_vix_dataset.csv')

In [411]:
# Compute returns of snp
vix_snp_df['S&P500_RETURN'] = vix_snp_df['S&P500'].pct_change()

# Compute the 30-day average of S&P500 return
vix_snp_df['S&P500_RETURN_AVG'] = vix_snp_df['S&P500_RETURN'].rolling(window=30, min_periods=1).mean()

In [413]:
# use forward-fill to fill up average interest rate during the weekend / non-trading days
vix_snp_df.index = pd.to_datetime(vix_snp_df['Date'])
full_date = pd.date_range(start=vix_snp_df.index.min(),
                         end=vix_snp_df.index.max(),
                         freq='D')
vix_snp_df = vix_snp_df.reindex(full_date).ffill()

In [415]:
# Compute the VIX into ml_df
ml_df['VIX'] = np.nan
for idx in ml_df.index:
    curr_date = ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)
    ml_df.loc[idx, 'VIX'] = vix_snp_df.loc[curr_date, 'VIX']

In [416]:
# Compute the SNP into ml_df
ml_df['SNP'] = np.nan
for idx in ml_df.index:
    curr_date = ml_df.loc[idx, 'FILING_DATE'] + pd.Timedelta(days=21)
    ml_df.loc[idx, 'SNP'] = vix_snp_df.loc[curr_date, 'S&P500_RETURN_AVG']

### Additional Features

Add Population Feature

In [421]:
# import population
file_path = 'demographic_dataset/population_dataset/population_dataset.xlsx'
population_df = pd.read_excel(file_path)

# Add the population to ml_df
ml_df['POPULATION'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    curr_year = ml_df.loc[idx, 'FILING_DATE'].year
    mask = (population_df['Year'] == curr_year)
    ml_df.loc[idx, 'POPULATION'] = population_df.loc[mask, curr_state].iloc[0]

Add House Pricing Index (HPI) Feature

In [423]:
# import HPI index
file_path = 'macroeconomy_dataset/hpi_dataset/hpi_dataset.xlsx'
hpi_df = pd.read_excel(file_path)

# Add the HPI index to ml_df
ml_df['HPI'] = np.nan
for idx in ml_df.index:
    curr_state = ml_df.loc[idx, 'STATEORCOUNTRY']
    curr_year = ml_df.loc[idx, 'FILING_DATE'].year
    curr_quarter = ml_df.loc[idx, 'FILING_DATE'].quarter
    mask = (hpi_df['yr'] == curr_year) & (hpi_df['period'] == curr_quarter) & (hpi_df['place_id'] == curr_state)
    ml_df.loc[idx, 'HPI'] = hpi_df.loc[mask, 'index_sa'].iloc[0]

Add Covid Indicator Feature

In [427]:
# Define Covid period
covid_start = pd.Timestamp('2020-03-01')
covid_end = pd.Timestamp('2021-06-30')

# Add covid indicator feature
ml_df['IS_COVID'] = np.nan
for idx in ml_df.index:
    if (ml_df.loc[idx, 'FILING_DATE'] >= covid_start) & (ml_df.loc[idx, 'FILING_DATE'] <= covid_end):
        ml_df.loc[idx, 'IS_COVID'] = 1
    else:
        ml_df.loc[idx, 'IS_COVID'] = 0

Add Post-SEC Regulation Change Feature

In [429]:
# Define the date when Reg CF regulation was amended
change_date = pd.Timestamp('2021-03-26')

# Add post-change feature
ml_df['IS_POSTREGCHANGE'] = np.nan
for idx in ml_df.index:
    if ml_df.loc[idx, 'FILING_DATE'] >= change_date:
        ml_df.loc[idx, 'IS_POSTREGCHANGE'] = 1
    else:
        ml_df.loc[idx, 'IS_POSTREGCHANGE'] = 0

Add Cash Runway

In [434]:
# Compute monthly net burn rate
ml_df['NETBURNRATE'] = (ml_df['NETINCOME'] * -1)/12

# Compute cash runway (in month)
ml_df['CASHRUNWAY'] = np.arcsinh((ml_df['CASH']) / (ml_df['NETBURNRATE'] + 1e-10))

### Cleaning The ml_df Dataset (Part 2)

Only include CF offerings thst end between 2018 Q1 and 2023 Q4 (This had been completed previously).

Only include CF offerings that start between 2018 Q1 and 2023 Q4 (This is just completed below).

In [449]:
ml_df_2 = ml_df.copy()

In [451]:
# Ensure to only include CF offerings that start between 2018 Q1 and 2023 Q4
for file_number in ml_df_2.index:
    if ml_df_2.loc[file_number, 'FILING_DATE'] >= pd.Timestamp('2018-01-01'):
        pass
    else:
        ml_df_2.drop(index=file_number, inplace = True)

Only include relevant columns

In [453]:
ml_df.columns

Index(['Y', 'REVENUE', 'PREVIOUSREVENUE', 'ASSET', 'CASH', 'LONGTERMDEBT',
       'NETINCOME', 'SHORTTERMDEBT', 'ACCREC', 'COGS', 'MINIMUMSIZE',
       'TEXT_RAISEDCOMMISSION', 'TEXT_EQUITYCOMMISSION', 'COMMISSIONCIK',
       'FILING_DATE', 'FINALDEADLINEDATE', 'FINALOFFERINGAMOUNT',
       'SECURITYOFFEREDTYPE', 'SECURITYOFFEREDOTHERDESC',
       'PRICEDETERMINATIONMETHOD', 'DATEINCORPORATION', 'CIK',
       'ISSUERSIGNATURE', 'CURRENTEMPLOYEES', 'LEGALSTATUSFORM',
       'LEGALSTATUSOTHERDESC', 'JURISDICTIONORGANIZATION', 'STATEORCOUNTRY',
       'REVENUEGROWTH', 'DEBTTOASSET', 'WORKINGCAPITAL', 'ROA', 'GROSSPROFIT',
       'GROSSMARGIN', 'NETPROFITMARGIN', 'RAISEDCOMMISSION',
       'EQUITYCOMMISSION', 'PLATFORMPOPULARITY', 'DURATION',
       'COMPETITORSCOUNT', 'EXTRACTED_SECURITYOFFEREDOTHERDESC',
       'FINALSECURITYOFFEREDTYPE', 'IS_PRICEMETHOD', 'COMPANYAGE',
       'PASTCOMPANYSUCCESS', 'PASTCOMPANYFAILURE', 'PASTPERSONSUCCESS',
       'PASTPERSONFAILURE', 'ISDELAWARE', 'LOCA

In [455]:
# List of all relevant columns
relevant_columns = [
    'Y',
    'WORKINGCAPITAL','REVENUE','REVENUEGROWTH','DEBTTOASSET','ROA','ASSET','CASH','LONGTERMDEBT','NETINCOME','GROSSPROFIT','GROSSMARGIN','NETPROFITMARGIN','CASHRUNWAY',
    'UNEMPLOYMENTRATE','AVERAGEINTERESTRATE','INFLATIONRATE','EX_UNEMPLOYMENTRATE','EX_SPOTINTERESTRATE','EX_INFLATIONRATE','EPU','CONSUMERSENTIMENT','VIX','SNP','GINI','GPR','HPI',
    'CREATIVEWORKFORCE','RUCC','AGE','INCOMELEVEL','BACHELOREDUCATION', 'POPULATION',
    'MINIMUMSIZE','RAISEDCOMMISSION','EQUITYCOMMISSION','PLATFORMPOPULARITY','DURATION','COMPETITORSCOUNT', 'LOCALCOMPETITORSCOUNT', 'FINALOFFERINGAMOUNT','FINALSECURITYOFFEREDTYPE','IS_PRICEMETHOD','IS_COVID', 'IS_POSTREGCHANGE',
    'COMPANYAGE','PASTCOMPANYSUCCESS','PASTCOMPANYFAILURE','PASTPERSONSUCCESS','PASTPERSONFAILURE','CURRENTEMPLOYEES','ISDELAWARE'
]

ml_df_2 = ml_df_2.loc[:, relevant_columns]

Check for nulls in the features and then drop those null rows

In [459]:
# Check for nulls
ml_df_2.isnull().sum()

Y                             0
WORKINGCAPITAL                0
REVENUE                       0
REVENUEGROWTH                 0
DEBTTOASSET                   0
ROA                           0
ASSET                         0
CASH                          0
LONGTERMDEBT                  0
NETINCOME                     0
GROSSPROFIT                   0
GROSSMARGIN                   0
NETPROFITMARGIN               0
CASHRUNWAY                    0
UNEMPLOYMENTRATE              0
AVERAGEINTERESTRATE           0
INFLATIONRATE                 0
EX_UNEMPLOYMENTRATE           0
EX_SPOTINTERESTRATE           0
EX_INFLATIONRATE              0
EPU                           0
CONSUMERSENTIMENT             0
VIX                           0
SNP                           0
GINI                          0
GPR                           0
HPI                           0
CREATIVEWORKFORCE             0
RUCC                          0
AGE                           0
INCOMELEVEL                   0
BACHELOR

In [461]:
# Data points with missing 'MINIMUMSIZE' and 'REVENUEGROWTH' = np.nan (due to negative disclosed REVENUE) are removed
ml_df_2.dropna(inplace = True)

Export to CSV File

In [467]:
ml_df_2.to_csv('ml_features.csv')

In [465]:
ml_df_2

,Y,WORKINGCAPITAL,REVENUE,REVENUEGROWTH,DEBTTOASSET,ROA,ASSET,CASH,LONGTERMDEBT,NETINCOME,...,IS_PRICEMETHOD,IS_COVID,IS_POSTREGCHANGE,COMPANYAGE,PASTCOMPANYSUCCESS,PASTCOMPANYFAILURE,PASTPERSONSUCCESS,PASTPERSONFAILURE,CURRENTEMPLOYEES,ISDELAWARE
FILE_NUMBER,,,,,,,,,,,,,,,,,,,,,
020-23892,1.0,5960.00,90802.0,11.416448,0.000000,-0.082502,258379.00,2460.00,0.00,-21341.00,...,0.0,0.0,0.0,1112,0.0,0.0,0.0,0.0,1.0,0.0
020-23894,1.0,-13075.00,0.0,-8.987572,6.081844,-6.672245,60.00,60.00,0.00,-23705.00,...,1.0,0.0,0.0,4464,0.0,0.0,0.0,0.0,11.0,1.0
020-23896,1.0,-195005.00,30747.0,10.333580,0.628790,-1.060695,329742.00,2961.00,0.00,-419129.00,...,0.0,0.0,0.0,3373,1.0,0.0,1.0,0.0,1.0,0.0
020-23900,1.0,1195.00,0.0,0.000000,0.000000,-4.904443,1195.00,1195.00,0.00,-80591.00,...,0.0,0.0,0.0,2934,1.0,0.0,1.0,0.0,4.0,0.0
020-23904,1.0,35977.16,21487.2,1.457424,0.100068,-0.513261,38692.43,35977.16,3878.33,-20742.82,...,0.0,0.0,0.0,1756,0.0,0.0,0.0,1.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
020-33264,0.0,54225.00,607816.0,2.208979,0.632368,0.016843,94577.00,63463.00,54636.00,1593.00,...,1.0,0.0,1.0,903,0.0,0.0,0.0,0.0,10.0,0.0
020-33266,0.0,-120510.00,92456.0,11.434499,2.753577,-4.597427,17276.00,14545.00,0.00,-857050.00,...,1.0,0.0,1.0,1146,0.0,0.0,0.0,0.0,1.0,0.0
020-33272,1.0,89047.00,1880819.0,-0.240243,0.606216,0.194890,1808372.00,156207.00,1097494.00,354669.00,...,1.0,0.0,1.0,7510,1.0,0.0,1.0,0.0,25.0,0.0
